# Курс "Машиное обучение в бизнесе"

## Практическое задание урока 2. Профилирование пользователей. Сегментация: unsupervised learning (clustering, LDA/ARTM), supervised (multi/binary classification)

In [1]:
import pandas as pd

Наши новости

In [2]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [3]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


### 1. Получаем векторные представления новостей

In [5]:
#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [7]:
#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [10]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [11]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [12]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [13]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Possible nested set at position 39
  from ipykernel import kernelapp as app


Wall time: 53.8 s


In [16]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 13min 13s


Обучим нашу модель

In [17]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Запускаем обучение

In [21]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 1min 13s


In [22]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [23]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(2, 0.37435365),
 (8, 0.16872832),
 (9, 0.015734205),
 (10, 0.06472919),
 (13, 0.26765004),
 (20, 0.09178006)]

In [24]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: суд иск решение депутат рубль запрет признать
topic_1: дело гражданин убийство орган человек сотрудник задержать
topic_2: украина украинский путин станция это запуск сектор
topic_3: пациент тело врач лечение больница нож мужчина
topic_4: украина россия который это nn власть проверка
topic_5: китай компания продукция китайский население год спрос
topic_6: россия это российский год который страна правительство
topic_7: лётчик тур комплексный болгария рисунок хорватия трансляция
topic_8: научный миссия японский год университет южный физика
topic_9: газ восток турция турецкий необычный греция медведев
topic_10: год рубль который метод это египет nn
topic_11: фонд остров собственность бизнесмен сенатор кандидат диапазон
topic_12: сша американский операция пенсия риск общество северный
topic_13: журнал статья форум связанный автор обращение публикация
topic_14: ii орден король русь сергеев колебаться дворец
topic_15: год военный наука млрд армия который это
topic_16: млн год рост со

Напишем функцию, которая будет нам возвращать векторное представление новости

In [25]:
# функциz, которая будет возвращать векторное представление новости
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [26]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.0,0.000000,0.0,0.0,0.000000,0.160451,0.000000,0.045103,...,0.0,0.000000,0.083900,0.000000,0.000000,0.688210,0.0,0.015101,0.0,0.000000
1,4896,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.080818,...,0.0,0.000000,0.000000,0.000000,0.000000,0.532120,0.0,0.000000,0.0,0.000000
2,4897,0.0,0.0,0.374435,0.0,0.0,0.000000,0.000000,0.000000,0.168239,...,0.0,0.000000,0.000000,0.000000,0.000000,0.091957,0.0,0.000000,0.0,0.000000
3,4898,0.0,0.0,0.000000,0.0,0.0,0.148063,0.000000,0.012085,0.000000,...,0.0,0.018917,0.040286,0.368137,0.202323,0.067058,0.0,0.000000,0.0,0.000000
4,4899,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.379128,0.165338,0.0,0.028472,0.0,0.319678


Мы получили вектора наших новостей.

Следующий шаг - векторные представления пользователей:

In [27]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

### Модификация кода функции get_user_embedding

2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
3. Повторить п.2, но используя уже не медиану, а max
4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

In [29]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding(user_articles_list, metric=np.mean):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = metric(user_vector, 0)
    return user_vector

In [30]:
get_user_embedding(user_articles_list, np.mean)

array([0.01512587, 0.05942968, 0.10240056, 0.        , 0.12980492,
       0.06248417, 0.15389864, 0.00290547, 0.00270484, 0.00170723,
       0.03446378, 0.00734166, 0.05991783, 0.        , 0.        ,
       0.05145842, 0.        , 0.10673775, 0.07508964, 0.09349767,
       0.        , 0.        , 0.        , 0.015863  , 0.01668485])

In [31]:
get_user_embedding(user_articles_list, np.median)

array([0.        , 0.        , 0.06076192, 0.        , 0.12585833,
       0.02707472, 0.06854127, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.03061871, 0.        , 0.        ,
       0.        , 0.        , 0.1075395 , 0.06028709, 0.07114133,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [32]:
get_user_embedding(user_articles_list, np.max)

array([0.09075519, 0.33644941, 0.26220104, 0.        , 0.2543689 ,
       0.26637694, 0.4947229 , 0.01743282, 0.01622906, 0.01024335,
       0.19220102, 0.04404998, 0.16749683, 0.        , 0.        ,
       0.30875054, 0.        , 0.22001214, 0.20691943, 0.27757534,
       0.        , 0.        , 0.        , 0.095178  , 0.10010912])

In [33]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [36]:
# Функция обучения модели и расчета метрик
def get_metrics(metric):
    
    #получим эмбединги для всех пользователей
    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, metric), 1)])
    user_embeddings.columns = [f'topic_{i}' for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(25)]]
    X = pd.merge(user_embeddings, target, 'left')
    
    #разделим данные на train/test
    X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                        X['churn'], shuffle=True, train_size=0.7, random_state=42)

    logreg = LogisticRegression()
    #обучим
    logreg.fit(X_train, y_train)
    
    #наши прогнозы для тестовой выборки
    preds = logreg.predict_proba(X_test)[:, 1]
    
    #рассчитаем Precision, Recall, F_score, Roc auc
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    roc_auc = roc_auc_score(y_test, preds)
    ix = np.argmax(fscore)
    
    return roc_auc, precision[ix], recall[ix], fscore[ix]

#### mean

In [37]:
m = list(get_metrics(np.mean))
print(f'roc_auc = {m[0]:.3f}, precision = {m[1]:.3f}, recall = {m[2]:.3f}, fscore = {m[3]:.3f}')

roc_auc = 0.964, precision = 0.757, recall = 0.762, fscore = 0.760


#### median

In [38]:
m = list(get_metrics(np.median))
print(f'roc_auc = {m[0]:.3f}, precision = {m[1]:.3f}, recall = {m[2]:.3f}, fscore = {m[3]:.3f}')

roc_auc = 0.973, precision = 0.787, recall = 0.772, fscore = 0.780


#### max

In [39]:
m = list(get_metrics(np.max))
print(f'roc_auc = {m[0]:.3f}, precision = {m[1]:.3f}, recall = {m[2]:.3f}, fscore = {m[3]:.3f}')

roc_auc = 0.981, precision = 0.833, recall = 0.827, fscore = 0.830


#### Таблица, сравнивающая качество 3 разных методов получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

In [40]:
metrics = {'mean': np.mean, 'median': np.median, 'max': np.max}
info = pd.DataFrame(columns=metrics.keys())

In [41]:
for metric, value in metrics.items():
    print(metric, value)
    roc_auc, precision, recall, fscore = get_metrics(value)
    info.loc[f'roc_auc', metric] = roc_auc
    info.loc[f'precision', metric] = precision
    info.loc[f'recall', metric] = recall
    info.loc[f'fscore', metric] = fscore
    
info

mean <function mean at 0x0000015086A9A9D8>
median <function median at 0x0000015086BF5438>
max <function amax at 0x0000015086A97798>


,mean,median,max
roc_auc,0.963997,0.973345,0.98061
precision,0.757282,0.787375,0.832787
recall,0.762215,0.771987,0.827362
fscore,0.75974,0.779605,0.830065


#### Выводы

При методе получения эмбедингов пользователей по max значения всех метрик выше, чем в методах по mean, median.
В нашей модели качественные величины мы преобразовывали в количественные, но их суть от этого не изменилась. Какое-либо качество или есть или нет, среднего не дано. Поэтому max является более эффективным способом.